# Adjusting, Renaming and Restructuring

In this tutorial we will cover how to modify the structure of an MRIO system.
This includes renaming sectors, regions and Y categories,
as well as restructuring (extracting and merging) satellite accounts.

We use the small test MRIO system that is included in the pymrio package for all examples.
The method work the same for any MRIO system.

In [1]:
import pymrio

In [2]:
mrio = pymrio.load_test()

We can use several functions to get a quick overview over the MRIO system [(see the Exploring MRIO tutorial for more examples)](explore.ipynb).

In [3]:
mrio.get_sectors()

Index(['food', 'mining', 'manufactoring', 'electricity', 'construction',
       'trade', 'transport', 'other'],
      dtype='object', name='sector')

In [4]:
mrio.get_regions()

Index(['reg1', 'reg2', 'reg3', 'reg4', 'reg5', 'reg6'], dtype='object', name='region')

In [5]:
mrio.get_Y_categories()

Index(['Final consumption expenditure by households',
       'Final consumption expenditure by non-profit organisations serving households (NPISH)',
       'Final consumption expenditure by government',
       'Gross fixed capital formation', 'Changes in inventories',
       'Changes in valuables', 'Export'],
      dtype='object', name='category')

A list of available satellite accounts can be obtained by

In [6]:
mrio.get_extensions()

<generator object IOSystem.get_extensions at 0x7fd30563a350>

this returns a generator over all extensions, to just get the names which can be used to loop over the extensions. To just get the names:

In [7]:
list(mrio.get_extensions())

['factor_inputs', 'emissions']

## Renaming

### Manual renaming

All the names returned are literally just names and can easily be renamed.
To rename sectors or regions just pass a dictionary with the new names:

In [8]:
mrio.rename_regions({"reg1": "REGION A", "reg2": "REGION B"})

In [9]:
mrio.get_regions()

Index(['REGION A', 'REGION B', 'reg3', 'reg4', 'reg5', 'reg6'], dtype='object', name='region')

Renaming sectors or Y categories works the same way

In [10]:
mrio.rename_sectors({"mining": "dwarf business"})
mrio.rename_Y_categories({"Final consumption expenditure by households": "fin_house"})

In [11]:
mrio.get_sectors()

Index(['food', 'dwarf business', 'manufactoring', 'electricity',
       'construction', 'trade', 'transport', 'other'],
      dtype='object', name='sector')

In [12]:
mrio.get_Y_categories()

Index(['fin_house',
       'Final consumption expenditure by non-profit organisations serving households (NPISH)',
       'Final consumption expenditure by government',
       'Gross fixed capital formation', 'Changes in inventories',
       'Changes in valuables', 'Export'],
      dtype='object', name='category')

### Renaming based on included classifications synonyms

Some MRIOs come with a selections of names which can be used for renaming.
These can be obtained via 'get_classification', passing the name of the IO (pass None to get a list of available classifications).

In [13]:
mrio_class = pymrio.get_classification(mrio_name="test")

The classification data contains different names and aggregation levels for the sectors and final demand categories. The easiest way to explore the classification data is by using the autocomplete functionality. Depending on the editor your are using this might work with typing mrio_class.sectors. pressing Tab or Ctrl-Space. The same works for mrio_class.finaldemand.

Alternatively it is possible to just print the underlying dataframes with (similar for finaldemand)

In [14]:
mrio_class.sectors

,TestMrioNumber,TestMrioName,TestMrioCode,Type
0,1,food,sec1,eat
1,2,mining,sec2,dig
2,3,manufactoring,sec3,build
3,4,electricity,sec4,util
4,5,construction,sec5,build
5,6,trade,sec6,marg
6,7,transport,sec7,marg
7,8,other,sec8,misc


This can be used to generate dictionaries for renaming the sectors of the MRIO, eg with:

In [15]:
conv_dict = mrio_class.get_sector_dict(
    mrio_class.sectors.TestMrioName, mrio_class.sectors.TestMrioCode
)
conv_dict

{'food': 'sec1',
 'mining': 'sec2',
 'manufactoring': 'sec3',
 'electricity': 'sec4',
 'construction': 'sec5',
 'trade': 'sec6',
 'transport': 'sec7',
 'other': 'sec8'}

This can then be used for renaming the sectors:

In [16]:
mrio = pymrio.load_test()

In [17]:
mrio.rename_sectors(conv_dict)
mrio.get_sectors()

Index(['sec1', 'sec2', 'sec3', 'sec4', 'sec5', 'sec6', 'sec7', 'sec8'], dtype='object', name='sector')

In the mrio_class.sectors you will also find an entry 'Type' which
represents a many to one mapping.
This can be used for aggregating the
MRIO and is explained [in the aggregation tutorial](aggregation_examples.ipynb#Aggregation-by-renaming).

# Extracting satellite accounts

Each extension/satellite account of the MRIO contains a method 'extract'.
This takes an index and extracts the corresponding satellite account entries.
The index can either be obtained manually, or one can use the contains/match/fullmatch methods of the extension to get the index.

Below we extract all extensions rows from the emission satallite account which contain the word 'air'.

In [18]:
air_emission_index = mrio.emissions.contains("air")
air_accounts_dict = mrio.emissions.extract(index=air_emission_index)

The result is a dictionary with the extracted satellite accounts. The keys are the names of the satellite accounts and the values are the extracted satellite accounts.

In [19]:
air_accounts_dict.keys()

dict_keys(['F', 'F_Y', 'unit'])

Alternatively, one can automatically convert the extracted satellite accounts to a new MRIO extension.

In [21]:
air_accounts_extension = mrio.emissions.extract(
    index=air_emission_index, return_type="extension"
)
print(air_accounts_extension)

Extension Emissions_extracted with parameters: name, F, F_Y, unit


The extract method can also be applied over all extensions.
This work well in combinaton with the contains/match/fullmatch methods of the extension.

Satellite accounts can be extracted from the MRIO system
by using the extension_extract method.
This method takes the name of the satellite account as first argument
and a list of regions as second argument.
The regions can be given as a list of region names or as a list of region indices.
The method returns a new MRIO system with the extracted satellite account.

In the somehow derived example below we extract entries accross
all accounts that contain the word 'Added' or the letter 2.

In [22]:
cross_accounts_index = mrio.extension_contains("Added|2")

We can also extract all data from the satellite accounts with these indexes.

In [23]:
cross_accounts_dict = mrio.extension_extract(cross_accounts_index)

This returns a dictionary of dictionaries with the extracted satellite accounts.

In [24]:
cross_accounts_dict.keys()

dict_keys(['Factor Inputs', 'Emissions'])

In [25]:
cross_accounts_dict["Emissions"].keys()

dict_keys(['F', 'F_Y', 'unit'])

Again, we can also build a new MRIO extension from the extracted satellite accounts.

This can be either one per extension (by passing return_type="extension")

In [26]:
cross_accounts_extension = mrio.extension_extract(
    cross_accounts_index, return_type="extension"
)

Or by passing any other string to return_type, which will be used as the new name of the extension.

In [27]:
cross_accounts_all_new = mrio.extension_extract(
    cross_accounts_index, return_type="new_name"
)

In [28]:
print(cross_accounts_all_new)

Extension new_name with parameters: name, F, F_Y, unit


Index values levels and names of the new extension are based on the combination
of the original extensions index levels.
These are filled for nan for entries which did not have them before.

In [29]:
cross_accounts_all_new.get_rows()

MultiIndex([(   'Value Added',     nan),
            ('emission_type2', 'water')],
           names=['indicator', 'compartment'])

Both extract methods, extension_extract and extract, can also be used to extract only
a subset of the available dataframe accounts. This can be done by passing a list of
dataframe names to the dataframes parameter.

In [30]:
only_fys = mrio.extension_extract(
    cross_accounts_index, dataframes=["F_Y"], include_empty=True
)

/home/konstans/proj/EXIOBASE/pymrio/pymrio/core/mriosystem.py:1691: UserWarning: Not all requested dataframes are available in Factor Inputs
  warnings.warn(


In [31]:
only_fys.keys()
only_fys["Emissions"].keys()

dict_keys(['F_Y'])